## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://pypi.org/project/rank-bm25/

In [71]:
path = '../04-filtrage/output/'
acteur = 'chum'
tag = ''

if tag:
    csv_file = acteur + '_' + tag + '_candidate-terms.csv'

else:
    csv_file = acteur + '_candidate-terms.csv'

### **Lire le vocabulaire** (termes retenus au prétraitement)

In [72]:
from pandas import *

with open(path+csv_file, encoding='utf-8') as f:
    csv = read_csv(f).drop(columns = ['Unnamed: 0'])
    # On va traiter les apostrophes et les parenthèses avec le underscore ("_") ; le tokenizer de sklearn ne les aime pas 
    csv['Terme formatté'] = csv["Terme"].apply(lambda x: str(x).replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_"))

csv

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté
0,chum,centre de recherche,NOM PRP NOM,1307,1152,-inf,False,False,centre de recherche
1,chum,recherche du chum,NOM PRP:det NOM,1140,1077,-inf,False,False,recherche du chum
2,chum,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,1065,-inf,False,False,centre de recherche du chum
3,chum,axe de recherche,NOM PRP NOM,961,959,-inf,False,False,axe de recherche
4,chum,chum axe,NOM NOM,932,932,-inf,False,False,chum axe
...,...,...,...,...,...,...,...,...,...
179,chum,santé professeur,NOM NOM,32,61,101.993026,False,False,santé professeur
180,chum,renseignements nécessaires,NOM ADJ,32,32,529.177964,False,False,renseignements nécessaires
181,chum,conférence du crchum,NOM PRP:det NOM,32,27,341.659072,False,False,conférence du crchum
182,chum,centre d'excellence,NOM PRP NOM,31,19,393.089776,False,False,centre d_excellence


In [73]:
vocabulaire = [t.lower() for t in list(csv['Terme formatté'])]

In [74]:
print('On a un vocabulaire de {} formes.'.format(len(vocabulaire)))

On a un vocabulaire de 184 formes.


In [75]:
vocabulaire

['centre de recherche',
 'recherche du chum',
 'centre de recherche du chum',
 'axe de recherche',
 'chum axe',
 'cabinet du médecin',
 'service d_urgence',
 'mesures appropriées',
 'assistance immédiate',
 'besoin d_assistance',
 'besoin d_assistance immédiate',
 'intérêts de recherche',
 'investigateur centre',
 'département de médecine',
 'pubmed recherche',
 'carrefour de l_innovation',
 'chercheur régulier',
 'évaluation en santé',
 'conférences du crchum',
 'axe de recherche carrefour',
 'recherche carrefour',
 'recherche cancer',
 'faculté de médecine',
 'axe de recherche immunopathologie',
 'recherche immunopathologie',
 'axe de recherche cancer',
 'chum professeur',
 'professeur adjoint',
 'chaire de recherche',
 'researchgate recherche',
 'recherche en santé',
 'professeur titulaire',
 'recherche du canada',
 'professeur agrégé',
 'axe de recherche cardiométabolique',
 'recherche cardiométabolique',
 'recherche clinique',
 'professeure adjointe',
 'centre hospitalier',
 'chai

### **Lire le corpus**

In [76]:
import os, shutil, re
from pathlib import Path
from os import path
from pandas import *

base_path = '../03-corpus/2-data/1-fr/'
if tag:
    base_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else:
    base_path = path.join(base_path, acteur +  '.csv')
        
with open(base_path, "r", encoding = "UTF-8") as f:
    data = read_csv(base_path, sep=',')
    text = data['text'].tolist()

In [77]:
text = text[:round(len(text))]

nb_docs = len(text)

print("On a donc un corpus de {} documents.".format(nb_docs))

On a donc un corpus de 1605 documents.


### **Nettoyage**

In [78]:
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    
punct = '[!#$%&•►*+,;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [re.sub(punct, ' ', t).replace("' ", "'" ) for t in corpus]
corpus = [re.sub(phones, ' STOP ', t) for t in corpus]
corpus = [re.sub(postals, ' STOP ', t) for t in corpus]
corpus = [t.replace("  ", " " ) for t in corpus]
corpus = [t.replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_") for t in corpus]


On va commencer par utiliser le CountVectorizer pour valider que l'implémentation de sklearn arrive bien au même compte que nous 

In [79]:
from nltk.tokenize import RegexpTokenizer
tokenizex = RegexpTokenizer("\w+'|\w+-\w+|\(|\)|\w+")

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer = tokenizex.tokenize, vocabulary=vocabulaire, ngram_range=(2,8), token_pattern='"\w+'|\w+-\w+|\(|\)|\w+"')
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

features_names = vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(X.T.todense(), index=features_names, columns=corpus_index).transpose()

In [80]:
csv['TF (sklearn)'] = 0
for t in vocabulaire:
    csv.loc[csv['Terme formatté'] == t, 'TF (sklearn)'] = df[t].sum()


csv['DF (sklearn)'] = 0
for t in vocabulaire:
    freqdoc = len(df[df[t] != 0])
    csv.loc[csv['Terme formatté'] == t, 'DF (sklearn)'] = freqdoc
csv

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn)
0,chum,centre de recherche,NOM PRP NOM,1307,1152,-inf,False,False,centre de recherche,1303,1150
1,chum,recherche du chum,NOM PRP:det NOM,1140,1077,-inf,False,False,recherche du chum,1140,1077
2,chum,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,1065,-inf,False,False,centre de recherche du chum,1116,1065
3,chum,axe de recherche,NOM PRP NOM,961,959,-inf,False,False,axe de recherche,955,953
4,chum,chum axe,NOM NOM,932,932,-inf,False,False,chum axe,932,932
...,...,...,...,...,...,...,...,...,...,...,...
179,chum,santé professeur,NOM NOM,32,61,101.993026,False,False,santé professeur,32,32
180,chum,renseignements nécessaires,NOM ADJ,32,32,529.177964,False,False,renseignements nécessaires,32,32
181,chum,conférence du crchum,NOM PRP:det NOM,32,27,341.659072,False,False,conférence du crchum,32,27
182,chum,centre d'excellence,NOM PRP NOM,31,19,393.089776,False,False,centre d_excellence,31,19


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import RegexpTokenizer

#toknizer = RegexpTokenizer()

# max_df : ignore words that appear in 85% of documents, 
# min df:  ignore words that appear in less than 1% of documents 
# vocabulary = vocabulaire

# Sans utiliser le vocabulaire
# tfidf = TfidfVectorizer(min_df=0.1, stop_words=None, ngram_range=(2,4), max_df=0.85, use_idf=True)

# vocabulary = vocabulaire
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulaire, ngram_range=(2,8), lowercase=False, token_pattern="\w+'|\w+-\w+|\(|\)|\w+")
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [82]:
features_names = tfidf_vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfidf.T.todense(), index=features_names, columns=corpus_index).transpose()

In [83]:
terms_tfidf = {term: df[term].max() for term in df}
zeros = {term : value for term, value in terms_tfidf.items() if terms_tfidf[term] == 0}

In [84]:
zeros

{'assistance immédiate': 0.0}

In [85]:
len(zeros) / len(vocabulaire) *100

0.5434782608695652

In [86]:
terms_tfidf

{'centre de recherche': 1.0,
 'recherche du chum': 0.5847483589370545,
 'centre de recherche du chum': 0.5894284502746543,
 'axe de recherche': 0.2561005142642352,
 'chum axe': 0.259848714224763,
 'cabinet du médecin': 0.25141916654106905,
 'service d_urgence': 0.2515951794021979,
 'mesures appropriées': 0.2515951794021979,
 'assistance immédiate': 0.0,
 'besoin d_assistance': 0.2515951794021979,
 'besoin d_assistance immédiate': 0.2515951794021979,
 'intérêts de recherche': 0.5626870991213508,
 'investigateur centre': 0.35926237511604275,
 'département de médecine': 0.5712824138887379,
 'pubmed recherche': 0.3633949406295931,
 'carrefour de l_innovation': 1.0,
 'chercheur régulier': 0.448882294088451,
 'évaluation en santé': 0.9414296853239121,
 'conférences du crchum': 0.7839934777327471,
 'axe de recherche carrefour': 0.46310785123515363,
 'recherche carrefour': 0.46180062669660144,
 'recherche cancer': 0.5359999308422418,
 'faculté de médecine': 0.78932715268174,
 'axe de recherche

In [87]:
terms_weighted = DataFrame(terms_tfidf.items(), columns=['Terme formatté', 'TF-IDF'])

In [88]:
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

terms_weighted = terms_weighted.drop_duplicates(keep='first')
terms_weighted

,Terme formatté,TF-IDF
0,centre de recherche,1.000000
30,recherche en santé,1.000000
36,recherche clinique,1.000000
145,demande de service,1.000000
46,projets de recherche,1.000000
...,...,...
124,développement professionnel continu,0.251511
122,activité de développement professionnel continu,0.251511
5,cabinet du médecin,0.251419
71,conférenciers scientifiques de renom,0.238869


In [89]:
terms_weighted = pd.merge(csv, terms_weighted, on='Terme formatté').drop_duplicates(
  subset = ['Terme', 'Fréquence (TF)'],
  keep = 'first').reset_index(drop = True)

terms_weighted

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,chum,centre de recherche,NOM PRP NOM,1307,1152,-inf,False,False,centre de recherche,1303,1150,1.000000
1,chum,recherche du chum,NOM PRP:det NOM,1140,1077,-inf,False,False,recherche du chum,1140,1077,0.584748
2,chum,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,1065,-inf,False,False,centre de recherche du chum,1116,1065,0.589428
3,chum,axe de recherche,NOM PRP NOM,961,959,-inf,False,False,axe de recherche,955,953,0.256101
4,chum,chum axe,NOM NOM,932,932,-inf,False,False,chum axe,932,932,0.259849
...,...,...,...,...,...,...,...,...,...,...,...,...
179,chum,santé professeur,NOM NOM,32,61,101.993026,False,False,santé professeur,32,32,0.491119
180,chum,renseignements nécessaires,NOM ADJ,32,32,529.177964,False,False,renseignements nécessaires,32,32,0.495022
181,chum,conférence du crchum,NOM PRP:det NOM,32,27,341.659072,False,False,conférence du crchum,32,27,0.845541
182,chum,centre d'excellence,NOM PRP NOM,31,19,393.089776,False,False,centre d_excellence,31,19,0.691901


## **OKapi BM25**
https://hal.archives-ouvertes.fr/hal-00760158 

In [90]:
from rank_bm25 import BM25Okapi

In [91]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("\w+'|\w+-\w+|\(|\)|\w+")

bm25 = BM25Okapi([tokenizer.tokenize(t) for t in corpus])

In [92]:
vocabulaire = csv["Terme"].dropna().tolist()

vocabulaire

['centre de recherche',
 'recherche du chum',
 'centre de recherche du chum',
 'axe de recherche',
 'chum axe',
 'cabinet du médecin',
 "service d'urgence",
 'mesures appropriées',
 'assistance immédiate',
 "besoin d'assistance",
 "besoin d'assistance immédiate",
 'intérêts de recherche',
 'investigateur centre',
 'département de médecine',
 'pubmed recherche',
 "carrefour de l'innovation",
 'chercheur régulier',
 'évaluation en santé',
 'conférences du crchum',
 'axe de recherche carrefour',
 'recherche carrefour',
 'recherche cancer',
 'faculté de médecine',
 'axe de recherche immunopathologie',
 'recherche immunopathologie',
 'axe de recherche cancer',
 'chum professeur',
 'professeur adjoint',
 'chaire de recherche',
 'researchgate recherche',
 'recherche en santé',
 'professeur titulaire',
 'recherche du canada',
 'professeur agrégé',
 'axe de recherche cardiométabolique',
 'recherche cardiométabolique',
 'recherche clinique',
 'professeure adjointe',
 'centre hospitalier',
 'chai

In [93]:
#tokenizer = RegexpTokenizer(r"\w\'|\w+")
## Revoir ça
tokenized_queries = [tokenizer.tokenize(str(t)) for t in vocabulaire]

features_names = [t for t in set(vocabulaire)]
corpus_index = [corpus.index(n) for n in corpus]

tab = [bm25.get_scores(query) for query in tokenized_queries]
df = pd.DataFrame(tab, index=features_names, columns=corpus_index).transpose()

In [94]:
print(len(tokenized_queries))

tokenized_queries

184


[['centre', 'de', 'recherche'],
 ['recherche', 'du', 'chum'],
 ['centre', 'de', 'recherche', 'du', 'chum'],
 ['axe', 'de', 'recherche'],
 ['chum', 'axe'],
 ['cabinet', 'du', 'médecin'],
 ['service', "d'", 'urgence'],
 ['mesures', 'appropriées'],
 ['assistance', 'immédiate'],
 ['besoin', "d'", 'assistance'],
 ['besoin', "d'", 'assistance', 'immédiate'],
 ['intérêts', 'de', 'recherche'],
 ['investigateur', 'centre'],
 ['département', 'de', 'médecine'],
 ['pubmed', 'recherche'],
 ['carrefour', 'de', "l'", 'innovation'],
 ['chercheur', 'régulier'],
 ['évaluation', 'en', 'santé'],
 ['conférences', 'du', 'crchum'],
 ['axe', 'de', 'recherche', 'carrefour'],
 ['recherche', 'carrefour'],
 ['recherche', 'cancer'],
 ['faculté', 'de', 'médecine'],
 ['axe', 'de', 'recherche', 'immunopathologie'],
 ['recherche', 'immunopathologie'],
 ['axe', 'de', 'recherche', 'cancer'],
 ['chum', 'professeur'],
 ['professeur', 'adjoint'],
 ['chaire', 'de', 'recherche'],
 ['researchgate', 'recherche'],
 ['recherche'

In [95]:
df

,santé mentale,développement professionnel,cancer professeur,chercheur professionnel,immunopathologie professeure,professeur associé,professeure agrégée,conférences du crchum,équipes de recherche,réponse immunitaire,...,système nerveux,chercheurs du centre de recherche,faculté de médecine,axe cardiométabolique,demande de soumission,calendrier des conférences,axe immunopathologie,système immunitaire,département de neurosciences,département de chirurgie
0,6.776433,6.942478,12.027358,5.084880,2.344867,2.906058,0.000000,0.000000,0.0,0.0,...,5.084880,0.0,0.0,5.084880,0.0,0.000000,0.0,2.906058,1.691553,0.0
1,6.106438,6.873731,11.639253,4.765521,2.363569,3.169246,0.000000,0.000000,0.0,0.0,...,4.765521,0.0,0.0,4.765521,0.0,0.489400,0.0,3.169246,1.340916,0.0
2,4.255505,4.576788,7.643210,4.255505,1.189084,2.198620,0.000000,0.000000,0.0,0.0,...,4.255505,0.0,0.0,6.715588,0.0,0.447138,0.0,2.198620,0.000000,0.0
3,4.327229,7.695336,10.180746,4.327229,2.485411,3.368107,0.000000,0.000000,0.0,0.0,...,4.327229,0.0,0.0,4.327229,0.0,0.000000,0.0,3.368107,0.000000,0.0
4,7.579414,7.305247,12.745642,5.440394,2.139020,3.027208,0.000000,0.000000,0.0,0.0,...,5.440394,0.0,0.0,5.440394,0.0,0.000000,0.0,3.027208,2.139020,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,5.738953,5.679680,9.187661,5.738953,1.217735,2.230973,1.846806,0.000000,0.0,0.0,...,5.738953,0.0,0.0,5.738953,0.0,0.300218,0.0,3.517206,0.000000,0.0
1276,5.228641,6.024139,9.501944,5.228641,2.136652,2.136652,1.750835,1.135644,0.0,0.0,...,5.228641,0.0,0.0,5.228641,0.0,0.000000,0.0,3.368506,0.000000,0.0
1277,5.900570,6.691830,10.181412,5.900570,2.035521,2.245321,1.387357,0.000000,0.0,0.0,...,5.900570,0.0,0.0,5.900570,0.0,0.000000,0.0,3.539827,0.000000,0.0
1278,5.870239,7.093994,10.556084,5.870239,2.032488,2.653357,2.032488,0.000000,0.0,0.0,...,5.870239,0.0,0.0,5.870239,0.0,0.000000,0.0,6.588458,0.000000,0.0


In [96]:
#df.to_csv(base_path + titre + '_matrice-OkapiBM25.csv') # Si on veut avoir la matrice (mais le fichier peut être très volumineux)

In [97]:
terms_okapi = {term: df[term].max() for term in df}

In [98]:
terms_okapi

{'santé mentale': 9.931537629207597,
 'développement professionnel': 9.542457303954475,
 'cancer professeur': 15.594375393171125,
 'chercheur professionnel': 9.437181151042836,
 'immunopathologie professeure': 5.554786811243165,
 'professeur associé': 7.9932933331645994,
 'professeure agrégée': 9.996507400894972,
 'conférences du crchum': 4.646805282622914,
 'équipes de recherche': 5.435289120702219,
 'réponse immunitaire': 5.435289120702219,
 'projets de recherche': 6.474596789565531,
 'fonds de recherche': 9.437181151042836,
 'évaluation en santé': 3.8138112621508964,
 'directeur laboratoire': 4.466989384584167,
 'performance scientifique': 3.616268070909131,
 'modèles animaux': 12.953172926740109,
 'recherche cardiométabolique': 5.188151198293477,
 'biologie moléculaire': 7.868462144227192,
 'soins infirmiers': 10.196103367268599,
 'chaires de recherche': 12.015794745190679,
 'conférenciers scientifiques de renom': 5.979708592576452,
 'santé professeur': 5.514973324942659,
 'institu

In [99]:
tab = DataFrame(terms_okapi.items(), columns=['Terme', 'OkapiBM25'])
tab

,Terme,OkapiBM25
0,santé mentale,9.931538
1,développement professionnel,9.542457
2,cancer professeur,15.594375
3,chercheur professionnel,9.437181
4,immunopathologie professeure,5.554787
...,...,...
179,calendrier des conférences,1.779855
180,axe immunopathologie,6.783961
181,système immunitaire,9.558004
182,département de neurosciences,7.374593


In [100]:
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

tab

,Terme,OkapiBM25
39,chirurgiens du canada,19.096117
79,réunions hebdomadaires,18.163380
149,centre hospitalier de l'université,17.913826
58,activité de développement professionnel,16.670938
129,centre de recherche du centre,16.593544
...,...,...
137,professeur au département,3.530905
95,chercheurs du crchum,3.173988
128,recherche chirurgie,2.825409
179,calendrier des conférences,1.779855


In [101]:
terms_weighted

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,chum,centre de recherche,NOM PRP NOM,1307,1152,-inf,False,False,centre de recherche,1303,1150,1.000000
1,chum,recherche du chum,NOM PRP:det NOM,1140,1077,-inf,False,False,recherche du chum,1140,1077,0.584748
2,chum,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,1065,-inf,False,False,centre de recherche du chum,1116,1065,0.589428
3,chum,axe de recherche,NOM PRP NOM,961,959,-inf,False,False,axe de recherche,955,953,0.256101
4,chum,chum axe,NOM NOM,932,932,-inf,False,False,chum axe,932,932,0.259849
...,...,...,...,...,...,...,...,...,...,...,...,...
179,chum,santé professeur,NOM NOM,32,61,101.993026,False,False,santé professeur,32,32,0.491119
180,chum,renseignements nécessaires,NOM ADJ,32,32,529.177964,False,False,renseignements nécessaires,32,32,0.495022
181,chum,conférence du crchum,NOM PRP:det NOM,32,27,341.659072,False,False,conférence du crchum,32,27,0.845541
182,chum,centre d'excellence,NOM PRP NOM,31,19,393.089776,False,False,centre d_excellence,31,19,0.691901


In [102]:
tab = pd.merge(terms_weighted, tab, on="Terme")
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

In [103]:
base_path = '../05-transformation/'

In [104]:
if tag:
    file_path = base_path + acteur + '_' + tag + '_weighting_OKapiBM25.csv'

else: 
    file_path = base_path + acteur  + '_weighting_OKapiBM25.csv'
tab.to_csv(file_path)

In [105]:
tab

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF,OkapiBM25
111,chum,chirurgiens du canada,NOM PRP:det NOM,46,46,615.737658,False,False,chirurgiens du canada,46,46,0.422580,19.096117
67,chum,réunions hebdomadaires,NOM ADJ,86,86,1585.372993,False,False,réunions hebdomadaires,86,86,0.382856,18.163380
44,chum,centre hospitalier de l'université,NOM ADJ PRP DET:ART NOM,115,88,1304.725425,False,False,centre hospitalier de l_université,115,88,0.716843,17.913826
126,chum,activité de développement professionnel,NOM PRP NOM ADJ,43,43,678.509886,False,False,activité de développement professionnel,43,43,0.251511,16.670938
75,chum,centre de recherche du centre,NOM PRP NOM PRP:det NOM,78,68,303.240884,False,False,centre de recherche du centre,78,68,0.459320,16.593544
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,chum,professeur au département,NOM PRP:det NOM,34,26,399.749714,False,False,professeur au département,34,26,0.712266,3.530905
151,chum,chercheurs du crchum,NOM PRP:det NOM,37,30,303.881974,False,False,chercheurs du crchum,37,30,1.000000,3.173988
170,chum,recherche chirurgie,NOM NOM,33,32,56.158435,False,False,recherche chirurgie,32,32,0.493759,2.825409
139,chum,calendrier des conférences,NOM PRP:det NOM,40,40,459.031480,False,False,calendrier des conférences,40,40,0.282236,1.779855
